In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import re
import string
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [21]:
#Quantidade maxima de páginas a serem extraidas
no_pages = 2

#Termo de busca digitado pelo user
search = "dan+brown"

def get_data(pageNo, searchTitle):  
    #Configuracao do BeatifulSoup
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    # MODELO DE URL DO SITE - https://www.travessa.com.br/Busca.aspx?d=1&bt=dan%20brown&cta=00&codtipoartigoexplosao=1&pag=2
    r = requests.get('https://www.travessa.com.br/Busca.aspx?d=1&bt='+str(searchTitle)+'&cta=00&codtipoartigoexplosao=1&pag='+ str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []

    #table = soup.find('table', attrs={'id':'gridBusca'})
    #print(table)
    #rows = table.find_all('tr')
    
    #Encontra no html todas as tables com essa class e pega seu conteúdo
    for d in soup.findAll('table', attrs={'style':'border-bottom-style:solid;border-bottom-width:1px;border-bottom-color:darkgrey;width:100%;'}):
        #print(d)
        name = d.find('a', attrs={'class':'buscaResultadoTitulo'})
        name = name.text
        name = string.capwords(name)
        name = re.split(' - ', name)
        name = name[0]


        #n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('span', attrs={'class':'buscaResultadoAutorProdutor'})
        #rating = d.find('span', attrs={'class':'4,1 de 5 estrelas'})
        #r = rating.find_all('span', aria-label=True)
        
        price = d.find('span', attrs={'class':'buscaResultadoPrecoPor'})
        price = price.text
        price = re.sub(r'Por:', '', price)

        imgUrl = d.find('img')['src']
        
        siteUrl = d.find('a')['href']

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(name)
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        #if rating is not None:
            #print(rating.text)
        #    all1.append(rating.text)
        #else:
        #    all1.append('-1')

        #if users_rated is not None:
            #print(price.text)
        #    all1.append(users_rated.text)
        #else:
        #    all1.append('0') 
        # 

        if imgUrl is not None:
            all1.append(imgUrl)
        else:
            all1.append("invalid-image")
        
        if siteUrl is not None:
            all1.append(siteUrl)
        else:
            all1.append("invalid-url-site")    

        if price is not None:
            #print(price.text)
            all1.append(price)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [22]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i, search))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Titulo','Autor', 'ImgUrl', 'SiteUrl', 'Preco'])
df.to_csv('travessa_products.csv', index=False, encoding='utf-8')

In [23]:
df = pd.read_csv("travessa_products.csv")
df.shape

(20, 5)

In [24]:
df.head(61)

,Titulo,Autor,ImgUrl,SiteUrl,Preco
0,O Simbolo Perdido,Dan Brown,https://img.travessa.com.br/livro/DT/01/01d85b...,https://www.travessa.com.br/o-simbolo-perdido-...,"R$ 33,92"
1,Sinfonia Dos Animais,Dan Brown,https://img.travessa.com.br/livro/DT/57/577b1a...,https://www.travessa.com.br/sinfonia-dos-anima...,"R$ 42,42"
2,Origem,Dan Brown,https://img.travessa.com.br/livro/DT/51/51b3c8...,https://www.travessa.com.br/origem-1-ed-2020/a...,"R$ 31,37"
3,O Codigo Da Vinci,Dan Brown,https://img.travessa.com.br/livro/DT/c0/c0352d...,https://www.travessa.com.br/o-codigo-da-vinci-...,"R$ 33,92"
4,Origem,Dan Brown,https://img.travessa.com.br/livro/DT/87/87277f...,https://www.travessa.com.br/origem-1-ed-2017/a...,"R$ 46,67"
5,Ponto De Impacto,Dan Brown,https://img.travessa.com.br/livro/DT/de/de0b2d...,https://www.travessa.com.br/ponto-de-impacto-1...,"R$ 46,67"
6,Fortaleza Digital,Dan Brown,https://img.travessa.com.br/livro/DT/ec/ecbe4c...,https://www.travessa.com.br/fortaleza-digital-...,"R$ 49,90"
7,Anjos E Demonios: A Primeira Aventura De Rober...,Dan Brown,https://img.travessa.com.br/livro/DT/0f/0ff506...,https://www.travessa.com.br/anjos-e-demonios-a...,"R$ 54,90"
8,Inferno,Dan Brown,https://img.travessa.com.br/livro/DT/10/103d4d...,https://www.travessa.com.br/inferno-1-ed-2013/...,"R$ 54,90"
9,Guia Para A Chave De Salomao De Dan Brown,Greg Taylor,https://img.travessa.com.br/livro/DT/35/35d6d9...,https://www.travessa.com.br/guia-para-a-chave-...,"R$ 59,90"
